In [3]:
import numpy as np
from sklearn import preprocessing
import pandas as pd


data = pd.read_csv("road sign prediction/Raw data/train.csv")
label_encoder = preprocessing.LabelEncoder()
data["DetectedCamera"] = label_encoder.fit_transform(data["DetectedCamera"])
target =  label_encoder.fit_transform(data["SignFacing (Target)"])
features_train = data[["DetectedCamera","AngleOfSign","SignAspectRatio","SignWidth","SignHeight"]]
features_train.head()

,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,2,195,1.02,46,45
1,2,203,1.09,59,54
2,0,26,0.96,104,108
3,2,199,0.81,38,47
4,2,208,0.93,54,58


In [5]:
#feature selection test
#data.drop("DetectedCamera",axis=1,inplace=True)
data.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target)
0,2c9180975a056a64015a1e0a52e57021,2,195,1.02,46,45,Rear
1,2c9180975a056a64015a1e17b32171e4,2,203,1.09,59,54,Rear
2,2c9180975a056a64015a1de4deb16bd5,0,26,0.96,104,108,Front
3,2c9180975a056a64015a1de4deb16bdd,2,199,0.81,38,47,Rear
4,2c9180975a056a64015a1de4deb16bd6,2,208,0.93,54,58,Rear


In [2]:
# Utility function to report best scores
from time import gmtime, strftime
def report(results, n_top=10):
    print("Report generated at {}".format(strftime("%d-%m-%Y %H:%M:%S")))
    print("")
    
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.6f} (std: {1:.6f}),time:{2:.3f}".format(
                results['mean_test_score'][candidate]+100,
                results['std_test_score'][candidate],
                results["mean_fit_time"][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [10]:
# this cell uses scipy optimise to optimise the continueous variables:
#'colsample_bytree', 'subsample', 'learning_rate', "reg_lambda"
# TODO implement a random search cv along with the scipy optimise algorithm

from sklearn.model_selection import cross_val_score,KFold
from scipy.optimize import minimize
from xgboost import XGBClassifier
from sklearn import model_selection

seed = 7
kfold = KFold(n_splits=5, random_state=None)
print(kfold)

def return_model_score(hyperparam_list):
    """
    :param hyperparam_list: 
    :return: returns the -mean score
    """
    hyperparam_dict = convert_list_to_params_dict(hyperparam_list)
    clf = XGBClassifier(**hyperparam_dict)
    cv_results = cross_val_score(clf,features_train, target, cv=kfold, scoring="neg_log_loss")
    mean_score = cv_results.mean()
    print(100+mean_score)
    return -mean_score

n_iterators = 132
mx_dpth =3

def convert_list_to_params_dict(hyperparam_list):
    """
    :param hyperparam_list: 
    :return: a dictionary with all the parameters
    """
    hyperparam_list = hyperparam_list.tolist()
    param_names = ['colsample_bytree', 'subsample', 'learning_rate', "reg_lambda",'n_estimators',"max_depth"]
    hyperparam_list.append(n_iterators)
    hyperparam_list.append(mx_dpth)
    return dict(zip(param_names, hyperparam_list))
    

bnds = ((0.1, 0.99),(0.1, 0.99),(0.1, 0.99),(0.1, 3.99))
x0 = [0.9207857677, 0.694497031849, 0.10928828638,  0.9815238592]
res = minimize(return_model_score, bounds=bnds, method='SLSQP',x0=x0,tol=1e-15)
best_score = 100-res.fun
res,best_score

KFold(n_splits=5, random_state=None, shuffle=False)


99.8910389887


99.8910389887


99.8910389887


99.8910389887


...

99.8910448424


99.8910448428


99.8908785129


99.8908861241


ValueError: Objective function must return a scalar

In [25]:
special_clf = XGBClassifier(**convert_list_to_params_dict(res.x))
special_clf.fit(features_train, target)

XGBClassifier(base_score=0.5, colsample_bylevel=1,
       colsample_bytree=0.7837743770093988, gamma=0,
       learning_rate=0.12112962321954159, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=117, nthread=-1,
       objective='multi:softprob', reg_alpha=0,
       reg_lambda=0.7103106817210079, scale_pos_weight=1, seed=0,
       silent=True, subsample=0.5761633533904281)

In [9]:
from time import time
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.externals import joblib
from time import strftime


# build a classifier
clf = XGBClassifier()


# specify parameters and distributions to sample from
param_dist = {#'min_samples_split':sp_randint(60,400),
              "learning_rate":sp_uniform(0.01,0.4),
              "gamma":sp_uniform(0.001,30),
              "n_estimators":sp_randint(60,270),
              "max_depth":sp_randint(3,12),
              "colsample_bytree":np.random.uniform(0.3,0.99,1000),
               'subsample': np.random.uniform(0.3,0.999,1000),
               'reg_lambda': sp_uniform(0.3,2)}

# run randomized search
n_iter_search = 250
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,scoring='neg_log_loss',cv=10
                                   ,n_jobs=-1)

start = time()
random_search.fit(features_train, target)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

# activate this when fitting models

best_score =100+random_search.best_score_
current_time = strftime("%d-%b-%Y %H-%M-%S")
joblib.dump(random_search, 'road sign prediction/classifier pickle dump/road sign_{0}'
                           'pv score {1:.6f}.pkl'.format(current_time,
                                                     100+random_search.best_score_))

report(random_search.cv_results_)

RandomizedSearchCV took 5413.48 seconds for 250 candidates parameter settings.
Report generated at 24-06-2017 04:13:36

Model with rank: 1
Mean validation score: 99.893256 (std: 0.014888),time:33.348
Parameters: {'colsample_bytree': 0.61595685609282425, 'n_estimators': 249, 'subsample': 0.73674410815315228, 'learning_rate': 0.064685535546596271, 'max_depth': 4, 'reg_lambda': 2.2256403579971868}

Model with rank: 2
Mean validation score: 99.892845 (std: 0.014779),time:9.864
Parameters: {'learning_rate': 0.13558441984083477, 'colsample_bytree': 0.88511043957300073, 'subsample': 0.69324362078939383, 'n_estimators': 98, 'max_depth': 3, 'reg_lambda': 1.8644479712734272}

Model with rank: 3
Mean validation score: 99.892795 (std: 0.014477),time:19.749
Parameters: {'colsample_bytree': 0.47413350146586042, 'n_estimators': 159, 'subsample': 0.90966708580999667, 'learning_rate': 0.17061293203137462, 'max_depth': 3, 'reg_lambda': 0.99097603441002136}

Model with rank: 4
Mean validation score: 99.8

In [13]:
report(random_search.cv_results_,n_top=10)

Report generated at 24-06-2017 04:18:56

Model with rank: 1
Mean validation score: 99.893256 (std: 0.014888),time:33.348
Parameters: {'colsample_bytree': 0.61595685609282425, 'n_estimators': 249, 'subsample': 0.73674410815315228, 'learning_rate': 0.064685535546596271, 'max_depth': 4, 'reg_lambda': 2.2256403579971868}

Model with rank: 2
Mean validation score: 99.892845 (std: 0.014779),time:9.864
Parameters: {'learning_rate': 0.13558441984083477, 'colsample_bytree': 0.88511043957300073, 'subsample': 0.69324362078939383, 'n_estimators': 98, 'max_depth': 3, 'reg_lambda': 1.8644479712734272}

Model with rank: 3
Mean validation score: 99.892795 (std: 0.014477),time:19.749
Parameters: {'colsample_bytree': 0.47413350146586042, 'n_estimators': 159, 'subsample': 0.90966708580999667, 'learning_rate': 0.17061293203137462, 'max_depth': 3, 'reg_lambda': 0.99097603441002136}

Model with rank: 4
Mean validation score: 99.892700 (std: 0.015198),time:9.165
Parameters: {'colsample_bytree': 0.68309561360

In [4]:
import xgboost as xgb
dtrain = xgb.DMatrix(data = features_train,label =  target)
params = {
    #'objective':"multi:logistic",
    'eval_metric':'logloss',
    'eta':0.025,
    'max_depth':6,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'min_child_weight':5
    
}

In [5]:
bst = xgb.cv(params, dtrain, num_boost_round=1000, early_stopping_rounds=40,nfold=5,verbose_eval=10)

[0]	train-logloss:0.586638+0.000313649	test-logloss:0.587116+0.000883757


[10]	train-logloss:-0.347239+0.0333613	test-logloss:-0.341038+0.0327104


[20]	train-logloss:-15.0139+0.243256	test-logloss:-14.9119+0.15621


[30]	train-logloss:-15.5377+0.0676258	test-logloss:-15.4383+0.146231


[40]	train-logloss:-15.6488+0.0522609	test-logloss:-15.5245+0.156276


...

[130]	train-logloss:-15.8341+0.0420306	test-logloss:-15.6707+0.181564


[140]	train-logloss:-15.848+0.0436167	test-logloss:-15.6744+0.179567


[150]	train-logloss:-15.8534+0.0425516	test-logloss:-15.6734+0.178279


[160]	train-logloss:-15.8597+0.0441791	test-logloss:-15.6682+0.180884


[170]	train-logloss:-15.868+0.0435192	test-logloss:-15.6654+0.1831


In [6]:
bst

,test-logloss-mean,test-logloss-std,train-logloss-mean,train-logloss-std
0,0.587116,0.000884,0.586638,0.000314
1,0.497023,0.024949,0.496132,0.024792
2,0.406002,0.024178,0.404673,0.024163
3,0.321926,0.027455,0.320024,0.028437
4,0.230131,0.027472,0.227822,0.028060
5,0.158567,0.018942,0.155659,0.020248
6,0.061336,0.019182,0.057666,0.020387
7,-0.033642,0.020388,-0.037938,0.021876
8,-0.130063,0.024121,-0.134856,0.024925
9,-0.235108,0.025776,-0.240927,0.026865


#Now we process the test data

In [6]:
data_test = pd.read_csv("road sign prediction/Raw data/test.csv")
#label_encoder = preprocessing.LabelEncoder()

data_test["DetectedCamera"]=label_encoder.transform(data_test["DetectedCamera"])
test_features = data_test[["DetectedCamera","AngleOfSign","SignAspectRatio","SignWidth","SignHeight"]]

test_features.head()
dtest = xgb.DMatrix(data=test_features)

In [58]:
#temporary
#from sklearn.externals import joblib
#grid = joblib.load('C:/Users/user/PycharmProjects/Machine learning competition/'
#                   'road sign prediction/classifier pickle dump/'
#                   'road sign_21-Jun-2017 23-10-08 pv score 99.890652 pb score 99.90083.pkl')
#100+grid.best_score_
#grid.best_params_

{'colsample_bytree': 0.72938240846461122,
 'learning_rate': 0.15358118809559876,
 'max_depth': 4,
 'n_estimators': 83,
 'reg_lambda': 1.1388004088252832,
 'subsample': 0.80859704015852019}

In [18]:
bst_train = xgb.XGBClassifier.fit(params,
                      dtrain, 
                      num_boost_round=1000)

XGBoostError: b'value 0for Parameter num_class should be greater equal to 1'

In [8]:
# if the test data has been loaded this is where we continue
#features_test_culled = model.transform(test_features)
#probablities = grid.predict_proba(features_test_culled)
probablities = bst_train.predict_proba(dtest)

#probablities = special_clf.predict_proba(test_features)

submission_data = pd.DataFrame(data = probablities,
                               columns=["Front", "Left", "Rear", "Right"])
submission_data.insert(0,"Id",data_test["Id"])

from time import strftime
current_time = strftime("%d-%b-%Y %H-%M-%S")
submission_data.to_csv("road sign prediction/output_data/road_sign_detection"
                       "_results_{0} pv score {1:.6f} .csv".format(current_time,
                                                              best_score)
                       ,index=False,float_format='%0.15f')

AttributeError: 'Booster' object has no attribute 'predict_proba'

In [ ]:
#submit online